In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gc

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

import math
import numpy as np
import pickle

pd.options.display.max_columns = 500

def clip_y(y):
    return [min(y, 20) for y in y]

clip_y(np.array([1, 2, 3, 4, 56, 77, 20]))

[1, 2, 3, 4, 20, 20, 20]

In [2]:
df = pd.read_pickle("data/processed/sales_monthly_wide.pickle")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424124 entries, 0 to 424123
Columns: 312 entries, shop_id to min_item_price_33
dtypes: Int16(204), float64(102), int64(3), object(3)
memory usage: 597.0+ MB


In [3]:
print(df.columns.values)

['shop_id' 'item_id' 'item_name' 'item_category_id' 'item_category_name'
 'shop_name' 'sum_item_cnt_month_0' 'sum_item_cnt_month_1'
 'sum_item_cnt_month_2' 'sum_item_cnt_month_3' 'sum_item_cnt_month_4'
 'sum_item_cnt_month_5' 'sum_item_cnt_month_6' 'sum_item_cnt_month_7'
 'sum_item_cnt_month_8' 'sum_item_cnt_month_9' 'sum_item_cnt_month_10'
 'sum_item_cnt_month_11' 'sum_item_cnt_month_12' 'sum_item_cnt_month_13'
 'sum_item_cnt_month_14' 'sum_item_cnt_month_15' 'sum_item_cnt_month_16'
 'sum_item_cnt_month_17' 'sum_item_cnt_month_18' 'sum_item_cnt_month_19'
 'sum_item_cnt_month_20' 'sum_item_cnt_month_21' 'sum_item_cnt_month_22'
 'sum_item_cnt_month_23' 'sum_item_cnt_month_24' 'sum_item_cnt_month_25'
 'sum_item_cnt_month_26' 'sum_item_cnt_month_27' 'sum_item_cnt_month_28'
 'sum_item_cnt_month_29' 'sum_item_cnt_month_30' 'sum_item_cnt_month_31'
 'sum_item_cnt_month_32' 'sum_item_cnt_month_33' 'min_item_cnt_month_0'
 'min_item_cnt_month_1' 'min_item_cnt_month_2' 'min_item_cnt_month_3'
 'mi

In [4]:
df[["shop_id", "item_id"]].drop_duplicates()

,shop_id,item_id
0,0,30
1,0,31
2,0,32
3,0,33
4,0,35
...,...,...
424119,59,22154
424120,59,22155
424121,59,22162
424122,59,22164


In [5]:
df_test = pd.read_csv("data/source/test.csv")
df_test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [6]:
shop_item = df[["shop_id", "item_id"]].drop_duplicates().join(
    df_test.set_index(["shop_id", "item_id"]), on=["shop_id", "item_id"], how="outer"
)

shop_item["ID"] = shop_item["ID"].fillna(-1).astype("int")

shop_item

,shop_id,item_id,ID
0,0,30,-1
1,0,31,-1
2,0,32,-1
3,0,33,-1
4,0,35,-1
...,...,...,...
424123,45,9500,214188
424123,45,16007,214189
424123,45,18027,214194
424123,45,16188,214196


In [7]:
items = pd.read_csv("data/source/items.csv")
cats = pd.read_csv("data/source/item_categories.csv")

items = items.join(cats.set_index("item_category_id"), how="left", on="item_category_id")

items

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD
...,...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31,Игры PC - Цифра
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54,Книги - Цифра
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49,Книги - Методические материалы 1С
22168,Яйцо для Little Inu,22168,62,"Подарки - Гаджеты, роботы, спорт"


In [8]:
shops = pd.read_csv("data/source/shops.csv")

shops.head(5)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
# add item and shop metadata
shop_item = shop_item.join(items.set_index("item_id"), how="left", on="item_id")
shop_item = shop_item.join(shops.set_index("shop_id"), on="shop_id", how="left")

shop_item

,shop_id,item_id,ID,item_name,item_category_id,item_category_name,shop_name
0,0,30,-1,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,31,-1,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,32,-1,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,33,-1,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
4,0,35,-1,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
...,...,...,...,...,...,...,...
424123,45,9500,214188,В НЕКОТОРОМ ЦАРСТВЕ сб. м/ф (Регион),40,Кино - DVD,"Самара ТЦ ""ПаркХаус"""
424123,45,16007,214189,Настольная игра 7 чудес,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус"""
424123,45,18027,214194,Резинки для плетения Коллекция Средневековья Н...,70,Подарки - Сувениры (в навеску),"Самара ТЦ ""ПаркХаус"""
424123,45,16188,214196,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус"""


In [10]:
shop_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526920 entries, 0 to 424123
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   shop_id             526920 non-null  int64 
 1   item_id             526920 non-null  int64 
 2   ID                  526920 non-null  int64 
 3   item_name           526920 non-null  object
 4   item_category_id    526920 non-null  int64 
 5   item_category_name  526920 non-null  object
 6   shop_name           526920 non-null  object
dtypes: int64(4), object(3)
memory usage: 32.2+ MB


In [11]:
shop_item

,shop_id,item_id,ID,item_name,item_category_id,item_category_name,shop_name
0,0,30,-1,007: КООРДИНАТЫ «СКАЙФОЛЛ»,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,31,-1,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,32,-1,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,33,-1,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
4,0,35,-1,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
...,...,...,...,...,...,...,...
424123,45,9500,214188,В НЕКОТОРОМ ЦАРСТВЕ сб. м/ф (Регион),40,Кино - DVD,"Самара ТЦ ""ПаркХаус"""
424123,45,16007,214189,Настольная игра 7 чудес,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус"""
424123,45,18027,214194,Резинки для плетения Коллекция Средневековья Н...,70,Подарки - Сувениры (в навеску),"Самара ТЦ ""ПаркХаус"""
424123,45,16188,214196,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус"""


In [13]:
"""
Create df with historic sales features
"""
date_block_num_prediction_index = 33  # consider data only before this index

df_long = pd.read_csv("data/processed/sales_monthly.csv")
y = df_long.query(f"date_block_num == {date_block_num_prediction_index}")
y = y.groupby(["shop_id", "item_id"]).agg({"sum_item_cnt_day": "sum"}).reset_index()

join_key = ["shop_id","item_id"]
y = y.join(shop_item[join_key].set_index(join_key), on=join_key, how="outer").fillna(0)
y = y.sort_values(by=["shop_id", "item_id"])

#df_long = df_long.query(f"date_block_num < {date_block_num_prediction_index}")
#df_long = df_long.groupby(["shop_id", "item_id"])


display(df_long)
display(y)

,date_block_num,shop_id,item_id,sum_item_cnt_day,min_item_cnt_day,max_item_cnt_day,min_date,max_date,count_date,mean_item_price,max_item_price,min_item_price,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,6,1,2,3,31,4,221.0,221.0,221.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,3,1,1,3,28,3,347.0,347.0,347.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,1,1,1,31,31,1,247.0,247.0,247.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,1,1,1,31,31,1,221.0,221.0,221.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,2,1,1,13,31,2,128.5,130.0,127.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609119,33,59,22087,6,1,3,5,23,3,119.0,119.0,119.0,Элемент питания DURACELL LR03-BC2,83,Элементы питания,"Ярославль ТЦ ""Альтаир"""
1609120,33,59,22088,2,1,1,3,27,2,119.0,119.0,119.0,Элемент питания DURACELL LR06-BC2,83,Элементы питания,"Ярославль ТЦ ""Альтаир"""
1609121,33,59,22091,1,1,1,3,3,1,179.0,179.0,179.0,Элемент питания DURACELL TURBO LR 03 2*BL,83,Элементы питания,"Ярославль ТЦ ""Альтаир"""
1609122,33,59,22100,1,1,1,18,18,1,629.0,629.0,629.0,Энциклопедия Adventure Time,42,"Книги - Артбуки, энциклопедии","Ярославль ТЦ ""Альтаир"""


,shop_id,item_id,sum_item_cnt_day
31530,0,30,0.0
31530,0,31,0.0
31530,0,32,0.0
31530,0,33,0.0
31530,0,35,0.0
...,...,...,...
31530,59,22162,0.0
31530,59,22163,0.0
31530,59,22164,0.0
31530,59,22166,0.0


In [14]:
y.describe()

,shop_id,item_id,sum_item_cnt_day
count,526920.000000,526920.000000,526920.000000
mean,31.524011,11401.045764,0.134852
std,17.144342,6178.105355,3.595077
min,0.000000,0.000000,-1.000000
25%,18.000000,6134.000000,0.000000
50%,31.000000,11610.000000,0.000000
75%,47.000000,16555.000000,0.000000
max,59.000000,22169.000000,2253.000000


# Concept

In order to have a better prediciton for previously unseen shop, item combinations, we do a prediction based on shop_id, item_id, item_category_id and potentially sales data, depending on if we have data for 

In [65]:
df_long.groupby(["shop_id", "item_category_id"]).agg({
    "date_block_num": ["min", "max", "count"],
    "sum_item_cnt_day": ["sum"]
}).reset_index()

shop_id item_category_id date_block_num           sum_item_cnt_day
                                         min max count              sum
0          0                0              1   1     1                1
1          0                1              1   1     1                1
2          0                2              0   1    37              105
3          0                3              0   1     2               52
4          0                4              0   1    11               38
...      ...              ...            ...  ..   ...              ...
3266      59               75              0  33   325              815
3267      59               77              4  26     4                4
3268      59               79              1  33    19              345
3269      59               80             30  32     5               30
3270      59               83              0  33   106              534

[3271 rows x 6 columns]

In [20]:
df_cats = df_long[["shop_id", "item_category_id", "sum_item_cnt_day"]]

In [21]:
df_cats

,shop_id,item_category_id,sum_item_cnt_day
0,0,40,6
1,0,37,3
2,0,40,1
3,0,40,1
4,0,57,2
...,...,...,...
1609119,59,83,6
1609120,59,83,2
1609121,59,83,1
1609122,59,42,1


In [22]:
df_cats

,shop_id,item_category_id,sum_item_cnt_day
0,0,40,6
1,0,37,3
2,0,40,1
3,0,40,1
4,0,57,2
...,...,...,...
1609119,59,83,6
1609120,59,83,2
1609121,59,83,1
1609122,59,42,1


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X = df_cats[["shop_id", "item_category_id"]]
y = df_cats[["sum_item_cnt_day"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [60]:
%%time

from sklearn.metrics import mean_squared_error

model = RandomForestRegressor(
    n_estimators=15,
    max_depth=10
)

model.fit(X_train, y_train.values.reshape(-1))

CPU times: user 15 s, sys: 154 ms, total: 15.1 s
Wall time: 15.3 s


RandomForestRegressor(max_depth=10, n_estimators=15)

In [61]:
print(f"""
RMSE train prediction: {
    mean_squared_error(
        y_true=clip_y(y_train.values.reshape(-1)),
        y_pred=clip_y(model.predict(X_train)),
        squared=False
)}

RMSE train 0 prediction: {
    mean_squared_error(
        y_true=clip_y(y_train.values.reshape(-1)),
        y_pred=[0] * X_train.shape[0],
        squared=False
)}

RMSE train mean prediction: {
    mean_squared_error(
        y_true=clip_y(y_train.values.reshape(-1)),
        y_pred=[y_train.mean()] * X_train.shape[0],
        squared=False
)}


RMSE test prediction: {
    mean_squared_error(
        y_true=clip_y(y_test.values.reshape(-1)),
        y_pred=clip_y(model.predict(X_test)),
        squared=False
)}

RMSE test 0 prediction: {
    mean_squared_error(
        y_true=clip_y(y_test.values.reshape(-1)),
        y_pred=[0] * X_test.shape[0],
        squared=False
)}

RMSE test mean prediction: {
    mean_squared_error(
        y_true=clip_y(y_test.values.reshape(-1)),
        y_pred=[y_test.mean()] * X_test.shape[0],
        squared=False
)}
""")


RMSE train prediction: 2.38043149431825

RMSE train 0 prediction: 3.2816014792214276

RMSE train mean prediction: 2.5949778502047867


RMSE test prediction: 2.378151954842212

RMSE test 0 prediction: 3.2676308817007627

RMSE test mean prediction: 2.5806857091104525



In [62]:
y_test.mean()

sum_item_cnt_day    2.256889
dtype: float64

In [ ]:
"""
How many ID in the test set don't have any history?
"""